# Speed Skydiving Analysis and Scoring

## Environment setup

In [1]:
from ssscore import COURSE_START
from ssscore import COURSE_END
from ssscore import DEG_IN_RAD
from ssscore import VALID_MSL
from ssscore import YEAR_PATH_TAG

import dateutil.parser
import math
import os
import os.path
import shutil

import pandas as pd

import ssscore

## FlySight data sources

1. Copy the FlySight Tracks/YY-MM-dd directory of interest to the DATA_LAKE directory.
1. Make a list of all CSV files in the FlySight DATA_LAKE
1. Move the CSV to the DATA_SOURCE bucket directory

### Define the data lake and data source:

In [2]:
DATA_LAKE     = './data-lake'
DATA_SOURCE   = './data-sources/pr3d4t0r'

ssscore.updateFlySightDataSource(DATA_LAKE, DATA_SOURCE)

moved 0 CSV files from data lake [./data-lake] to data source [./data-sources/pr3d4t0r]


## Top speed and pitch analysis on a single data file

User selects a valid FlySight data path and file name.  Call the `calculateSpeedAndPitch(fileName)` function with this file name.  The function produces:

* Classical mean speed calculated from all the discrete speeds sampled within the course
* USPA calculation ds/dt where dt<sub>competitionTime</sub> = t<sub>end</sub>-t<sub>start</sub>,
  and s<sub>courseLength</sub> = |s<sub>start</sub>-s<sub>end</sub>| for the first and last heights
  reported within the course window.
* Flight pitch in degrees at max speed
* Max speed
* Min speed

All speeds are reported in km/h.  Michael Cooper, USPA National Speed Skydiving Championship judged, explained
the "USPA calculation" as straight ds/dt in an email message from 04.Sep.2018.

### Explanation of the code in calculateSpeedAndPitchFrom(fileName)

1. Discard all source entries outside of the 2,700 to 1,700 m AGL course
1. Calculate the mean speed for the current jump (actual sum(N)/N and ds/dt for s0, s1
1. Resolve the max speed and pitch within the course
1. Return the results in a Series object, for later inclusion in a data frame with results from multiple jumps

### Specify the FlySight data file to analyze:

In [3]:
FLYSIGHT_DATA_FILE = '20181027.142406.CSV'

In [4]:
def _discardDataOutsideCourse(fullFlightData):
    height          = fullFlightData['hMSL', '(m)']
    descentVelocity = fullFlightData['velD', '(m/s)']
    
    return fullFlightData[(height <= COURSE_START) & (height >= COURSE_END) & (descentVelocity >= 0.0)]


def _calculateCourseSpeedUsing(flightData):
    """
    Returns course speed as ds/dt, and competitionTime in seconds
    """
    # TODO: figure out why pd.to_datetime() barfs
    startTime = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[0])
    endTime   = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[-1])

    startCourse = flightData['hMSL', '(m)'].values[0]
    endCourse   = flightData['hMSL', '(m)'].values[-1]

    competitionTime = (endTime-startTime).total_seconds()
    courseLength    = abs(endCourse-startCourse)

    return courseLength/competitionTime, competitionTime


def maxHorizontalSpeedFrom(flightData, maxVerticalSpeed):
    velN = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velN', '(m/s)']
    velE = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velE', '(m/s)']
    
    return math.sqrt(velN**2+velE**2)   # R vector


def calculateSpeedAndPitchFor(fileName):
    """
    Accepts a file name to a FlySight data file.
    
    Returns a Series with the results of a speed skydiving jump.
    """
    flightData  = _discardDataOutsideCourse(pd.read_csv(fileName, header = [0, 1]))
    sampledVelD = flightData['velD', '(m/s)'].mean()
    courseVelD, \
    courseTime  = _calculateCourseSpeedUsing(flightData)
    maxSpeed    = flightData['velD', '(m/s)'].max()
    minSpeed    = flightData['velD', '(m/s)'].min()
    
    pitchR       = math.atan(maxSpeed/maxHorizontalSpeedFrom(flightData, maxSpeed))
    maxSpeedTime = flightData[flightData['velD', '(m/s)'] == maxSpeed]['time', 'Unnamed: 0_level_1'].values[0]
    
    skydiveResults = pd.Series(
                        [
                            fileName,
                            maxSpeedTime,
                            3.6*sampledVelD,  # km/h; 3,600 seconds, 1,000 meters
                            3.6*courseVelD,
                            pitchR/DEG_IN_RAD,
                            3.6*maxSpeed,
                            3.6*minSpeed,
                            courseTime,
                        ],
                        [
                            'fileName',
                            'maxSpeedTime',
                            'sampledSpeed',
                            'courseSpeed',
                            'pitch',
                            'maxSpeed',
                            'minSpeed',
                            'courseTime',
                        ])

    return skydiveResults


calculateSpeedAndPitchFor(os.path.join(DATA_SOURCE, FLYSIGHT_DATA_FILE))

fileName        ./data-sources/pr3d4t0r/20181027.142406.CSV
maxSpeedTime                        2018-10-27T21:47:50.20Z
sampledSpeed                                        387.738
courseSpeed                                          388.59
pitch                                               72.9377
maxSpeed                                            405.792
minSpeed                                            357.048
courseTime                                              9.2
dtype: object

## Listing FlySight data files with valid data

**UNDER CONSTRUCTION**
1. Generate the list of available data files
1. Discard FlySight warm up files by rejecting files without jump data (test:  minimal altitude changes)

In [ ]:
def listDataFilesIn(path):
    """
    Generate a sorted list of files available in a given path.
    Files names appear in reverse lexicographical order.
    """

    filesList = pd.Series([os.path.join(path, fileName) for fileName in sorted(os.listdir(path), reverse = True) if '.CSV' in fileName])
    
    return filesList;

In [ ]:
def hasValidJumpData(fileName):
    flightData = pd.read_csv(fileName, header = [0, 1])['hMSL', '(m)']
    
    return flightData.std() >= VALID_MSL

In [ ]:
def discardFlySightWarmUpFilesFrom(dataFiles):
    included = dataFiles.apply(hasValidJumpData)
    
    return pd.Series(dataFiles)[included]

In [ ]:
dataFiles = discardFlySightWarmUpFilesFrom(listDataFilesIn(DATA_SOURCE))
dataFiles